# **Crawling PTA Prodi Manajemen Abstrak Id.**

# **1. Crawling**

**Library**


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re, sys, time
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


**Base Url**

In [4]:
Base_Url = "https://pta.trunojoyo.ac.id/c_search/byprod"

**Function**

In [5]:
def get_max_page(prodi_id):
    url = f"{Base_Url}/{prodi_id}"
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")

    # cari tombol >> (last page)
    last_page = soup.select_one('ol.pagination a:contains("»")')
    if last_page and "href" in last_page.attrs:
        href = last_page["href"]
        # pecah URL -> ambil angka terakhir
        max_page = int(href.split("/")[-1])
        return max_page

    # fallback kalau pagination tidak ada
    return 1

In [6]:
def print_progress(prodi_id, prodi, current_page, total_pages):
    percent = (current_page / total_pages) * 100
    bar_length = 20
    filled_length = int(bar_length * current_page // total_pages)
    bar = '█' * filled_length + '-' * (bar_length - filled_length)
    sys.stdout.write(f'\r[{prodi_id}] {prodi} - Page {current_page}/{total_pages} [{bar}] {percent:.2f}%')
    sys.stdout.flush()
    if current_page == total_pages:
        sys.stdout.write('\n')

**Function All Data**

In [11]:
def pta_manajemen():
    start_time = time.time()
    prodi_id = 7  # ID Prodi Manajemen
    data = {
        "id": [],
        "penulis": [],
        "judul": [],
        "abstrak_id": [],
        "pembimbing_pertama": [],
        "pembimbing_kedua": [],
        "prodi": [],
    }

    max_page = get_max_page(prodi_id)
    print(f"🔍 Total halaman Prodi Manajemen: {max_page}")

    for j in range(1, max_page + 1):
        url = f"{Base_Url}/{prodi_id}/{j}"
        r = requests.get(url)
        soup = BeautifulSoup(r.content, "html.parser")
        jurnals = soup.select('li[data-cat="#luxury"]')

        isii = soup.select_one('div#begin')
        if not isii:
            continue
        prodi_full = isii.select_one('h2').text.strip()
        prodi = prodi_full.replace("Journal Jurusan ", "")

        for jurnal in jurnals:
            link_keluar = jurnal.select_one('a.gray.button')['href']

            # ambil ID dari link PTA
            id_match = re.search(r"/detail/(\d+)", link_keluar)
            pta_id = id_match.group(1) if id_match else None
            response = requests.get(link_keluar)
            soup1 = BeautifulSoup(response.content, "html.parser")
            isi = soup1.select_one('div#content_journal')

            judul = isi.select_one('a.title').text.strip()

            # Penulis
            penulis = isi.select_one('span:contains("Penulis")').text.split(' : ')[1]

            # Pembimbing Pertama
            pembimbing_pertama = isi.select_one('span:contains("Dosen Pembimbing I")').text.split(' : ')[1]

            # Pembimbing Kedua
            pembimbing_kedua = isi.select_one('span:contains("Dosen Pembimbing II")').text.split(' :')[1]

            paragraf = isi.select('p[align="justify"]')
            abstrak_id = paragraf[0].get_text(strip=True) if len(paragraf) > 0 else "N/A"

            data["id"].append(pta_id)
            data["penulis"].append(penulis)
            data["judul"].append(judul)
            data["pembimbing_pertama"].append(pembimbing_pertama)
            data["pembimbing_kedua"].append(pembimbing_kedua)
            data["abstrak_id"].append(abstrak_id)
            data["prodi"].append(prodi)

        # ✅ Perbaikan: panggil dengan 4 argumen
        print_progress(prodi_id, prodi, j, max_page)

    # Simpan ke CSV
    df = pd.DataFrame(data)
    output_path = "/content/drive/MyDrive/PPW/output/pta_manajemen.csv"
    df.to_csv(output_path, index=False, encoding="utf-8-sig")

    # Hitung durasi
    end_time = time.time()
    elapsed = int(end_time - start_time)
    jam, sisa = divmod(elapsed, 3600)
    menit, detik = divmod(sisa, 60)

    print("\n✅ Seluruh data Manajemen berhasil dikumpulkan!")
    print(f"📈 Total entri: {len(df)}")
    print(f"⏱️ Waktu eksekusi: {jam} jam {menit} menit {detik} detik")

    return df

In [12]:
# --- Eksekusi ---
if __name__ == "__main__":
    prodi_id = 7
    pages = get_max_page(prodi_id)
    confirm = input("Mau lanjut scraping semua data Manajemen? (y/n): ")
    if confirm.lower() == "y":
        df = pta_manajemen()
    else:
        print("❌ Scraping dibatalkan.")

Mau lanjut scraping semua data Manajemen? (y/n): y
🔍 Total halaman Prodi Manajemen: 207
[7] Manajemen - Page 207/207 [████████████████████] 100.00%

✅ Seluruh data Manajemen berhasil dikumpulkan!
📈 Total entri: 1031
⏱️ Waktu eksekusi: 0 jam 31 menit 34 detik


In [13]:
manajemen_path = "/content/drive/MyDrive/PPW/output/pta_manajemen.csv"
manajemen = pd.read_csv(manajemen_path)

manajemen

,id,penulis,judul,abstrak_id,pembimbing_pertama,pembimbing_kedua,prodi
0,80211100070,SATIYAH,PENGARUH FAKTOR-FAKTOR PELATIHAN DAN PENGEMBAN...,"ABSTRAK\r\nSatiyah, Pengaruh Faktor-faktor Pel...","Dra. Hj. S. Anugrahini Irawati, MM","Helmi Buyung Aulia,S,ST.SE,M.MT",Manajemen
1,90211200001,Faishal,ANALISIS PERSEPSI BRAND ASSOCIATION MENURUT PE...,Tujuan penelitian ini adalah untuk mengetahui ...,Nurita Andriani,Yustina Chrismardani,Manajemen
2,80211100050,Wahyu Kurniawan,PENGARUH GAYA KEPEMIMPINAN DEMOKRATIK TERHADAP...,NaN,"Dr. Dra. Hj. Iriani Ismail, MM","Dra. Hj. S. Anugrahini Irawati, MM",Manajemen
3,100211200002,Muhammad Zakaria Utomo,Pengukuran Website Quality Pada Situs Sistem A...,Aplikasi nyata pemanfaatan teknologi informasi...,"Dr. Ir. Nurita Andriani, MM","Nirma Kurriwati, SP, M.Si",Manajemen
4,80211100044,Hendri Wahyudi Prayitno,PENGARUH KEPEMIMPINAN DAN KOMPENSASI TERHADAP ...,Abstrak\r\nPenelitian ini menggunakan metode k...,"Dra. Hj. S Anugrahini Irawati, MM","Helmi Buyung Aulia,S,ST,SE,.MT",Manajemen
...,...,...,...,...,...,...,...
1026,160211100071,Husnul Hotimah,Analisis Cost Volume Profit Untuk Menentukan T...,ABSTRAK\nPenelitian ini bertujuan untuk menget...,"Hj. Evaliati Amaniyah, S.E., M.S.M.",NaN,Manajemen
1027,160211100291,Uswatun Hasanah,Pengaruh Pelatihan Dan Kompensasi Terhadap Pro...,"ABSTRAK\nUswatun Hasanah, 160211100291, Pengar...","Dr. Raden Mas Mochammad Wispandono S.E ., MS",NaN,Manajemen
1028,160211100064,ACH FATHONI,PERAN SERVICE PERFORMANCE DAN CLIMATE ORGANIZA...,ABSTRAK\nTujuan dari penelitian ini adalah unt...,"YUDHI PRASETYA MADA, S.E., M.M.",NaN,Manajemen
1029,160211100030,INTAN YULLIA NINGSIH,BAURAN PROMOSI PADA DEALER YAMAHA TRETAN MOTOR...,ABSTRAK\nPenelitian ini bertujuan: (1) Untuk m...,"DR. MOHAMMAD ARIEF, S.E., M.M.",NaN,Manajemen


In [14]:
manajemen.isna().sum().sum()

np.int64(104)

In [15]:
# cari index (nomor baris) yang ada NaN
nan_rows = manajemen[manajemen.isna().any(axis=1)].index

print("Baris yang mengandung NaN:", nan_rows.tolist())


Baris yang mengandung NaN: [2, 154, 310, 372, 373, 385, 527, 929, 930, 931, 932, 933, 934, 935, 936, 937, 938, 939, 940, 941, 942, 945, 946, 947, 948, 949, 951, 952, 953, 954, 956, 957, 958, 959, 960, 961, 962, 963, 964, 965, 966, 968, 969, 970, 971, 972, 973, 974, 976, 977, 979, 980, 981, 982, 983, 984, 985, 986, 987, 988, 989, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1000, 1001, 1002, 1004, 1005, 1006, 1007, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030]


# **2. Pre-Processing**

In [16]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 13.3 MB/s eta 0:00:00


In [17]:
import pandas as pd
import re
from collections import Counter
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

##**Sample Preprocessing**

In [18]:
sample_text = manajemen['abstrak_id'].dropna().iloc[0]
print("🔹 TEKS ASLI:\n", sample_text, "\n")

🔹 TEKS ASLI:
 ABSTRAK
Satiyah, Pengaruh Faktor-faktor Pelatihan dan Pengembangan Terhadap Produktivitas Kerja Dinas Kelautan dan Perikanan Bangkalan. Dibawah bimbingan Dra.Hj.S.Anugrahini Irawati,MM dan  Helmi Buyung Aulia,S,ST.SE,M.MT
Dalam upaya meningkatkan produktivitas kerja tidaklah mudah, oleh karena itu salah satu usaha agar produktivitas meningkat adalah menerapkan program pelatihan dan pengembangan sumber daya manusia (SDM) perlu dilaksanakan dalam instansi agar produktivitas yang tinggi dapat tercapai dengan meningkat kemampuan pegawai agar dapat bekerja secara efektif dan efisien. Dengan adanya pelatihan dan pengembnagan diharapkan pegawai juga mampu menyesuaikan diri dengan kebutuhan-kebutuhan baru atas sikap, tingkah laku keterampilan dan pengetahuan sesuai dengan tuntutan perubahan. Dengan adanya pelatihan dan pengembangan pegawai yang baik dapat mendukung terciptanya suasana kerja yang kondusif dalam instansi, sehingga dengan sendirinya produktivitasi kerja akan meningk

In [19]:
# --- 1. Case Folding ---
casefolded = sample_text.lower()
print("🔹 CASE FOLDING:\n", casefolded, "\n")

🔹 CASE FOLDING:
 abstrak
satiyah, pengaruh faktor-faktor pelatihan dan pengembangan terhadap produktivitas kerja dinas kelautan dan perikanan bangkalan. dibawah bimbingan dra.hj.s.anugrahini irawati,mm dan  helmi buyung aulia,s,st.se,m.mt
dalam upaya meningkatkan produktivitas kerja tidaklah mudah, oleh karena itu salah satu usaha agar produktivitas meningkat adalah menerapkan program pelatihan dan pengembangan sumber daya manusia (sdm) perlu dilaksanakan dalam instansi agar produktivitas yang tinggi dapat tercapai dengan meningkat kemampuan pegawai agar dapat bekerja secara efektif dan efisien. dengan adanya pelatihan dan pengembnagan diharapkan pegawai juga mampu menyesuaikan diri dengan kebutuhan-kebutuhan baru atas sikap, tingkah laku keterampilan dan pengetahuan sesuai dengan tuntutan perubahan. dengan adanya pelatihan dan pengembangan pegawai yang baik dapat mendukung terciptanya suasana kerja yang kondusif dalam instansi, sehingga dengan sendirinya produktivitasi kerja akan meni

In [20]:
# --- 2. Cleaning (hapus angka, tanda baca, karakter khusus) ---
cleaned = re.sub(r'[^a-zA-Z\s]', '', casefolded)
print("🔹 CLEANING:\n", cleaned, "\n")

🔹 CLEANING:
 abstrak
satiyah pengaruh faktorfaktor pelatihan dan pengembangan terhadap produktivitas kerja dinas kelautan dan perikanan bangkalan dibawah bimbingan drahjsanugrahini irawatimm dan  helmi buyung auliasstsemmt
dalam upaya meningkatkan produktivitas kerja tidaklah mudah oleh karena itu salah satu usaha agar produktivitas meningkat adalah menerapkan program pelatihan dan pengembangan sumber daya manusia sdm perlu dilaksanakan dalam instansi agar produktivitas yang tinggi dapat tercapai dengan meningkat kemampuan pegawai agar dapat bekerja secara efektif dan efisien dengan adanya pelatihan dan pengembnagan diharapkan pegawai juga mampu menyesuaikan diri dengan kebutuhankebutuhan baru atas sikap tingkah laku keterampilan dan pengetahuan sesuai dengan tuntutan perubahan dengan adanya pelatihan dan pengembangan pegawai yang baik dapat mendukung terciptanya suasana kerja yang kondusif dalam instansi sehingga dengan sendirinya produktivitasi kerja akan meningkat
tujuan penelitian 

In [21]:
# --- 3. Tokenisasi ---
tokens = cleaned.split()
print("🔹 TOKENISASI:\n", tokens, "\n")

🔹 TOKENISASI:
 ['abstrak', 'satiyah', 'pengaruh', 'faktorfaktor', 'pelatihan', 'dan', 'pengembangan', 'terhadap', 'produktivitas', 'kerja', 'dinas', 'kelautan', 'dan', 'perikanan', 'bangkalan', 'dibawah', 'bimbingan', 'drahjsanugrahini', 'irawatimm', 'dan', 'helmi', 'buyung', 'auliasstsemmt', 'dalam', 'upaya', 'meningkatkan', 'produktivitas', 'kerja', 'tidaklah', 'mudah', 'oleh', 'karena', 'itu', 'salah', 'satu', 'usaha', 'agar', 'produktivitas', 'meningkat', 'adalah', 'menerapkan', 'program', 'pelatihan', 'dan', 'pengembangan', 'sumber', 'daya', 'manusia', 'sdm', 'perlu', 'dilaksanakan', 'dalam', 'instansi', 'agar', 'produktivitas', 'yang', 'tinggi', 'dapat', 'tercapai', 'dengan', 'meningkat', 'kemampuan', 'pegawai', 'agar', 'dapat', 'bekerja', 'secara', 'efektif', 'dan', 'efisien', 'dengan', 'adanya', 'pelatihan', 'dan', 'pengembnagan', 'diharapkan', 'pegawai', 'juga', 'mampu', 'menyesuaikan', 'diri', 'dengan', 'kebutuhankebutuhan', 'baru', 'atas', 'sikap', 'tingkah', 'laku', 'ketera

In [22]:
# --- 4. Stopword Removal ---
stop_factory = StopWordRemoverFactory()
stopword = stop_factory.create_stop_word_remover()
no_stop = stopword.remove(" ".join(tokens))
print("🔹 STOPWORD REMOVAL:\n", no_stop.split(), "\n")

🔹 STOPWORD REMOVAL:
 ['abstrak', 'satiyah', 'pengaruh', 'faktorfaktor', 'pelatihan', 'pengembangan', 'produktivitas', 'kerja', 'dinas', 'kelautan', 'perikanan', 'bangkalan', 'dibawah', 'bimbingan', 'drahjsanugrahini', 'irawatimm', 'helmi', 'buyung', 'auliasstsemmt', 'upaya', 'meningkatkan', 'produktivitas', 'kerja', 'tidaklah', 'mudah', 'salah', 'satu', 'usaha', 'produktivitas', 'meningkat', 'menerapkan', 'program', 'pelatihan', 'pengembangan', 'sumber', 'daya', 'manusia', 'sdm', 'perlu', 'dilaksanakan', 'instansi', 'produktivitas', 'tinggi', 'tercapai', 'meningkat', 'kemampuan', 'pegawai', 'bekerja', 'efektif', 'efisien', 'adanya', 'pelatihan', 'pengembnagan', 'diharapkan', 'pegawai', 'mampu', 'menyesuaikan', 'diri', 'kebutuhankebutuhan', 'baru', 'atas', 'sikap', 'tingkah', 'laku', 'keterampilan', 'pengetahuan', 'sesuai', 'tuntutan', 'perubahan', 'adanya', 'pelatihan', 'pengembangan', 'pegawai', 'baik', 'dapat', 'mendukung', 'terciptanya', 'suasana', 'kerja', 'kondusif', 'instansi', '

In [23]:
# --- 5. Stemming ---
stem_factory = StemmerFactory()
stemmer = stem_factory.create_stemmer()
stemmed = stemmer.stem(no_stop)
print("🔹 STEMMING:\n", stemmed.split(), "\n")

🔹 STEMMING:
 ['abstrak', 'satiyah', 'pengaruh', 'faktorfaktor', 'latih', 'kembang', 'produktivitas', 'kerja', 'dinas', 'laut', 'ikan', 'bangkal', 'bawah', 'bimbing', 'drahjsanugrahini', 'irawatimm', 'helm', 'buyung', 'auliasstsemmt', 'upaya', 'tingkat', 'produktivitas', 'kerja', 'tidak', 'mudah', 'salah', 'satu', 'usaha', 'produktivitas', 'tingkat', 'terap', 'program', 'latih', 'kembang', 'sumber', 'daya', 'manusia', 'sdm', 'perlu', 'laksana', 'instansi', 'produktivitas', 'tinggi', 'capai', 'tingkat', 'mampu', 'pegawai', 'kerja', 'efektif', 'efisien', 'ada', 'latih', 'pengembnagan', 'harap', 'pegawai', 'mampu', 'sesuai', 'diri', 'kebutuhankebutuhan', 'baru', 'atas', 'sikap', 'tingkah', 'laku', 'terampil', 'tahu', 'sesuai', 'tuntut', 'ubah', 'ada', 'latih', 'kembang', 'pegawai', 'baik', 'dapat', 'dukung', 'cipta', 'suasana', 'kerja', 'kondusif', 'instansi', 'sendiri', 'produktivitas', 'kerja', 'tingkat', 'tuju', 'teliti', 'adalah', 'tahu', 'berapa', 'besar', 'pengaruh', 'faktorfaktor', 

##**Preprocessing keseluruhan**

In [24]:
# === Ambil kolom abstrak ===
abstrak_list = manajemen["abstrak_id"].dropna().astype(str).tolist()

In [25]:
# === 1. Siapkan stemmer & stopword ===
factory = StemmerFactory()
stemmer = factory.create_stemmer()

stop_factory = StopWordRemoverFactory()
stopwords = set(stop_factory.get_stop_words())

In [26]:
# === 2. Preprocessing & tokenisasi ===
processed_words = []
for text in abstrak_list:   # abstrak_list sudah berisi daftar abstrak
    text = text.lower()  # case folding
    text = re.sub(r'[^a-z\s]', ' ', text)  # hapus non-huruf
    tokens = text.split()  # tokenisasi
    tokens = [t for t in tokens if t not in stopwords]  # hapus stopword
    stemmed = [stemmer.stem(t) for t in tokens]  # stemming
    processed_words.extend(stemmed)  # gabungkan semua kata

In [27]:
# === 3. Hitung frekuensi kata ===
counter = Counter(processed_words)
sorted_freq = counter.most_common()  # otomatis urut descending

In [28]:
# === 4. Simpan ke CSV dengan indeks ===
freq_df = pd.DataFrame(sorted_freq, columns=["kata", "frekuensi"])
freq_df.index += 1  # mulai indeks dari 1
freq_df.to_csv("/content/drive/MyDrive/PPW/output/frekuensi_kata_indexed.csv",
               index=True, encoding="utf-8-sig", index_label="indeks")

print("✅ Frekuensi kata berhasil disimpan dengan indeks.")
print(freq_df.head(20))  # tampilkan 10 kata teratas

✅ Frekuensi kata berhasil disimpan dengan indeks.
          kata  frekuensi
1     pengaruh       5555
2        kerja       5479
3       teliti       4399
4     variabel       3580
5         guna       2964
6        usaha       2564
7        besar       2462
8   signifikan       2440
9          uji       2375
10    karyawan       2272
11       nilai       1912
12       hasil       1793
13    analisis       1461
14        tahu       1450
15     positif       1307
16      sampel       1178
17        data       1081
18      tunjuk       1074
19       putus       1040
20        tuju       1037


In [29]:
preprocess_manajemen_path = "/content/drive/MyDrive/PPW/output/frekuensi_kata_indexed.csv"
preprocess_manajemen = pd.read_csv(preprocess_manajemen_path)

preprocess_manajemen


,indeks,kata,frekuensi
0,1,pengaruh,5555
1,2,kerja,5479
2,3,teliti,4399
3,4,variabel,3580
4,5,guna,2964
...,...,...,...
6035,6036,mixed,1
6036,6037,methods,1
6037,6038,eksplanatoris,1
6038,6039,sekuensial,1
